In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from torchtext.vocab import Vectors
from collections import deque
import collections

In [2]:
torch.manual_seed(1)

In [132]:
TEXT = torchtext.data.Field()
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path="", train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)
TEXT.build_vocab(train)
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=20, device=-1, bptt_len=35, repeat=False)
# Build the vocabulary with word embeddings
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

it = iter(train_iter)
batch = next(it) 

In [133]:
TEXT.vocab.vectors


 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0104 -0.1829  0.0761  ...  -0.1362 -0.2240 -0.0552
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-0.0421  0.0775  0.1566  ...   0.0535  0.3011 -0.1736
 0.0004 -0.2599  0.1088  ...   0.0129  0.2687 -0.1937
[torch.FloatTensor of size 10001x300]

In [134]:
class LSTM_Lang_Model(nn.Module):
    
    def __init__(self, embeddings, dropout_rate):
        super(LSTM_Lang_Model, self).__init__()
        self.embeddings = embeddings
        self.dropout_rate = dropout_rate
        self.lstm = nn.LSTM(input_size=embeddings.size(1), hidden_size=650, num_layers=2, dropout=dropout_rate)
        for w in rnn.lstm.parameters():
            w.data = torch.Tensor(w.size()).uniform_(-0.05, 0.05)
        self.dropout_o = nn.Dropout(p=dropout_rate)
        self.linear = nn.Linear(650, embeddings.size(0))
    
    def forward(self, text, hidden):
        input = Variable(self.embeddings[batch.text.data, :])
        output, hidden = self.lstm(input, hidden)
        self.output = output
        output = self.dropout_o(output)
        output = self.linear(output)
        probs = F.softmax(output.permute(2, 0, 1)).permute(1, 2, 0)
        return probs, hidden

In [136]:
loss_function = nn.NLLLoss()
rnn = LSTM_Lang_Model(embeddings=TEXT.vocab.vectors, dropout_rate=0.5)
optimizer = optim.SGD(rnn.parameters(), lr=0.1)

In [ ]:
train_iter.init_epoch()
hidden = None
for batch in train_iter:
    probs, hidden = rnn(batch.text, hidden)
    log_probs = torch.log(probs)
    loss = -log_probs.gather(2, batch.target.unsqueeze(2)).mean()
    loss.backward(retain_graph=True)
    for w in rnn.lstm.parameters():
        norm = w.grad.norm()
        if norm.data.numpy()[0] > 5:
            print('yes')
            w.grad = w.grad / norm
    optimizer.step()
    print(loss.data.numpy()[0])

9.21388
9.21179
9.20935
9.20507
9.19739
9.18771


In [110]:
hidden

(Variable containing:
 ( 0 ,.,.) = 
   2.3730e-02 -4.9902e-02 -4.3526e-02  ...   6.5671e-03  1.6821e-02  2.7558e-03
  -3.7160e-02 -1.7295e-02 -3.4130e-02  ...  -1.3602e-02  3.8968e-03 -1.4136e-02
   2.1907e-02 -3.9972e-02 -1.7203e-02  ...   8.0944e-04 -2.6492e-02  1.3127e-03
                  ...                   ⋱                   ...                
   2.2806e-02 -7.0939e-03 -3.9586e-02  ...  -2.0796e-02  7.2639e-03  4.8472e-03
   3.1893e-04 -4.3660e-02 -3.8795e-02  ...  -2.1925e-02  5.0662e-03 -4.0616e-03
  -3.7838e-03 -2.4350e-02 -6.6424e-02  ...  -8.2639e-03  3.2982e-02 -1.5917e-02
 
 ( 1 ,.,.) = 
  -1.8233e-02 -4.4981e-02  7.2307e-02  ...  -2.2500e-02  5.3280e-02 -5.3389e-02
  -2.4832e-02 -4.3759e-02  5.7221e-02  ...  -2.5631e-02  4.7603e-02 -4.6040e-02
  -2.5052e-02 -5.5347e-02  6.3033e-02  ...  -1.5348e-02  4.5850e-02 -4.5402e-02
                  ...                   ⋱                   ...                
  -3.0527e-02 -3.8360e-02  5.8920e-02  ...  -2.0339e-02  4.3557e-02 